In [1]:
import tensorflow as tf
import keras_nlp
import numpy as np
import os

/Users/emerickmiatti/Development/Formation IA/Projet/Emosens/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Charger le modèle BERT Base sauvegardé

In [2]:
MODEL_PATH = '../models/bert_base_20251213_170236.weights.h5'
PRESET = 'bert_base_en_uncased'
NUM_CLASSES = 28
SEQUENCE_LENGTH = 128

classifier = keras_nlp.models.BertClassifier.from_preset(
    PRESET,
    num_classes=NUM_CLASSES
)
classifier.preprocessor.sequence_length = SEQUENCE_LENGTH
classifier.load_weights(MODEL_PATH)
print('Modèle chargé !')

Modèle chargé !


/Users/emerickmiatti/Development/Formation IA/Projet/Emosens/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 404 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## 2. Tester l'inférence sur des exemples

In [3]:
LABEL_NAMES = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

def predict_emotions(text, threshold=0.5):
    logits = classifier.predict([text], verbose=0)
    probs = tf.nn.sigmoid(logits[0]).numpy()
    probs_percent = probs * 100
    detected = [(LABEL_NAMES[i], probs_percent[i]) for i in range(len(LABEL_NAMES)) if probs[i] >= threshold]
    detected.sort(key=lambda x: x[1], reverse=True)
    return detected, probs_percent

# Exemples
examples = [
    "I am so happy and excited!",
    "This is the worst experience ever. I'm so disappointed.",
    "I'm sad but also optimistic about the future."
]

for text in examples:
    emotions, probs = predict_emotions(text, threshold=0.4)
    print(f'Texte : {text}')
    if emotions:
        for emotion, prob in emotions:
            print(f'  • {emotion:15s} : {prob:.1f}%')
    else:
        print('  Aucune émotion détectée au-dessus du seuil.')
    print('-'*40)

Texte : I am so happy and excited!
  • excitement      : 97.0%
  • joy             : 93.4%
----------------------------------------
Texte : This is the worst experience ever. I'm so disappointed.
  • disappointment  : 96.4%
  • sadness         : 83.1%
  • embarrassment   : 75.5%
  • disgust         : 74.3%
  • annoyance       : 54.9%
----------------------------------------
Texte : I'm sad but also optimistic about the future.
  • sadness         : 95.7%
  • disappointment  : 63.9%
  • optimism        : 56.5%
  • grief           : 42.3%
----------------------------------------
